**Computational Health Laboratory Project, A.Y. 2021/2022**

**Authors:** Niko Dalla Noce, Alessandro Ristori, Andrea Zuppolini

**Project:** Starting fron one or more genes, extract from interaction databases the genes they interact with. Using the expanded gene set, perform pathway analysis and obtain all disease pathways in which the genes appear. Merge the pathways to obtain a larger graph. Perform further network analysis to extract central biomarkers and communities beyond pathways. Compute a distance between the initial gene set and the various pathways (diseases).

# **CHL Project, Community Analysis**

## **Colab setup**
Takes care of the project setup on Colab.

In [2]:
if 'google.colab' in str(get_ipython()):
    import subprocess
    out_clone = subprocess.run(["git", "clone", "https://github.com/nikodallanoce/ComputationalHealthLaboratory"], text=True, capture_output=True)
    print("{0}{1}".format(out_clone.stdout, out_clone.stderr))
    %cd ComputationalHealthLaboratory

## **Community analysis**
Find the communities inside the graph and look how the one in which the starting gene fell is connected to the various diseases.

In [3]:
import pandas as pd
import numpy as np
import networkx as nx

def intersection(lst1, lst2):
    inters = list()
    if not (len(lst1) == 0 or len(lst2) == 0):
        set1 = set(lst1)
        inters = [elem for elem in lst2 if elem in set1]
    return inters

We assume that you have already done pathway enrichment on notebook and network analysis on **0_Pathway_Enrichment** and **1_Network_Analysis** and, therefore, all the datasets needed here are available. If so, then load everything.

In [4]:
df_diseases = pd.read_csv("datasets/diseases_pathways.csv", sep=",", index_col=0)
protein_graph = nx.read_gpickle("datasets/protein_graph.gpickle")
diseases = dict()
for i, disease in df_diseases.iterrows():
    disease_genes = disease['Genes'].split(";")
    term = disease['Term']
    diseases[i] = {"name": term, "genes": disease_genes}

Find all the communities with the Louvain method (beware that is not deterministic) and discard those with only one node inside.

In [5]:
louvain_communities = list(nx.algorithms.community.louvain_communities(protein_graph))
communities = [community for community in louvain_communities if len(community) > 1]  # Discard those communities with one node

In [6]:
print("Number of communities: {0}".format(len(communities)))

Number of communities: 9


Compute the mean number of nodes of those communities kept after the pruning on their size.

In [7]:
def mean_size_communities(communities: list) -> float:
    mean_size = 0
    for community in communities:
        mean_size += len(community)

    mean_size /= len(communities)
    return mean_size

In [8]:
print("Mean size of communities: {0}".format(str(mean_size_communities(communities))))

Mean size of communities: 681.1111111111111


Compute the number of diseases in a community.

In [9]:
def diseases_in_community(protein_graph: nx.Graph, community: set) -> set:
    diseases_community = set()
    for protein in list(community):
        diseases_protein = protein_graph.nodes[protein]["diseases"]
        diseases_community.update(diseases_protein)

    return diseases_community

In [10]:
diseases_comm = diseases_in_community(protein_graph, communities[5])
print("Number of diseases in community {0}: {1}".format(5, len(diseases_comm)))

Number of diseases in community 5: 585


Let’s also see how many disease, on average, where linked to those communities with only one node.

In [11]:
def mean_diseases_communities_size_n(communities: list, protein_graph: nx.Graph, n: int = 1) -> float:
    mean_diseases = 0
    n_size_commmunities = 0
    for community in communities:
        community = list(community)
        if len(community) == n:
            protein = community[0]
            protein_diseases = protein_graph.nodes[protein]['diseases']
            n_size_commmunities += 1
            mean_diseases += len(protein_diseases)

    if n_size_commmunities == 0:
        print("There are no communities with {0} nodes".format(n))
    else:
        mean_diseases /= n_size_commmunities

    return mean_diseases

In [12]:
mean_diseases_one_node = mean_diseases_communities_size_n(louvain_communities, protein_graph)
print("Mean diseases for those communities with one node: {0}".format(str(mean_diseases_one_node)))

Mean diseases for those communities with one node: 2.2324127906976745


At last compute the mean number of nodes for the diseases.

In [13]:
def mean_genes_diseases(diseases: dict) -> float:
    mean_size = 0
    for _, disease in diseases.items():
        mean_size += len(disease['genes'])

    mean_size /= len(diseases.keys())
    return mean_size

In [14]:
print("Mean number of genes for disease: {0}".format(mean_genes_diseases(diseases)))

Mean number of genes for disease: 246.55857385398983


Let's see if the communities do not share any nodes.

In [15]:
def are_communities_distinct(communities: list) -> bool:
    for i, first_community in enumerate(communities):
        for j in range(i+1, len(communities)):
            second_community = communities[j]
            if len(intersection(first_community, second_community))>0:
                return False

    return True

In [16]:
are_communities_distinct(communities)

True

Compute how many proteins are shared between each community and disease.

In [17]:
def communities_metrics(communities: list, diseases: dict) -> pd.DataFrame:
    df_ranks = list()
    for i, community in enumerate(communities):
        tot_genes = dict()
        shared_genes = dict()
        for k, disease in diseases.items():
            genes = disease['genes']
            shared_genes_community = intersection(genes, community)
            tot_genes[k] = len(genes)
            shared_genes[k] = len(shared_genes_community)

        for j in range(len(tot_genes)):
            n_genes, n_shared_genes = tot_genes[j], shared_genes[j]
            if n_shared_genes > 1:
                df_ranks.append({"Community": i, "Disease": diseases[j]['name'], "Shared genes": n_shared_genes,
                                 "Disease genes": n_genes, "Community size": len(community)})

    df_ranks = pd.DataFrame(df_ranks)

    # Ratio of the shared genes (between community and disease pathway) and the number of genes in the disease
    df_ranks["Ratio disease"] = df_ranks['Shared genes'] / df_ranks['Disease genes']

    # Ratio of the shared genes (between community and disease pathway) and the size of the community
    df_ranks['Ratio community'] = df_ranks["Shared genes"] / df_ranks["Community size"]

    # Relevance of the  based on the previous computed metrics
    df_ranks["Relevance"] = df_ranks["Ratio disease"] * df_ranks["Ratio community"]
    return df_ranks

In [18]:
communities_rank = communities_metrics(communities, diseases)
communities_rank[communities_rank["Disease"]=="Tooth size discrepancy"]

,Community,Disease,Shared genes,Disease genes,Community size,Ratio disease,Ratio community,Relevance
517,0,Tooth size discrepancy,2,47,506,0.042553,0.003953,0.000168
1613,2,Tooth size discrepancy,3,47,895,0.063830,0.003352,0.000214
2190,3,Tooth size discrepancy,8,47,1084,0.170213,0.007380,0.001256
3071,5,Tooth size discrepancy,5,47,1081,0.106383,0.004625,0.000492
3617,6,Tooth size discrepancy,4,47,714,0.085106,0.005602,0.000477
4185,7,Tooth size discrepancy,12,47,522,0.255319,0.022989,0.005869
4743,8,Tooth size discrepancy,5,47,798,0.106383,0.006266,0.000667


In [19]:
communities_rank.head()

,Community,Disease,Shared genes,Disease genes,Community size,Ratio disease,Ratio community,Relevance
0,0,Mental and motor retardation,28,802,506,0.034913,0.055336,0.001932
1,0,Small head,15,470,506,0.031915,0.029644,0.000946
2,0,Cognitive delay,28,758,506,0.036939,0.055336,0.002044
3,0,Global developmental delay,32,854,506,0.037471,0.063241,0.002370
4,0,Mammary Neoplasms,167,1736,506,0.096198,0.330040,0.031749


In [20]:
communities_rank.to_csv("datasets/communities_metrics.csv")

In [21]:
def look_for_gene_community(protein: str, communities: list) -> int:
    for i, community in enumerate(communities):
        if protein in community:
            return i

    return -1

In [22]:
protein_community = "SON"
gene_community = look_for_gene_community(protein_community, communities)
if gene_community == -1:
    raise Exception("The gene {0} is not in one of the communities".format(protein_community))
else:
    print("The gene {0} is in community {1}".format(protein_community, gene_community))

The gene SON is in community 3


Knowing in which community our starting gene is, we can retrieve all the diseases inisde such community and rank them by their relevance.

In [47]:
# Retrieve the diseases inside the community in which the gene is in and keep those that share at least ten genes with the community
disease_rank = communities_rank[(communities_rank["Community"]==gene_community) &
                             (communities_rank["Shared genes"] > 10)].sort_values(by="Relevance", ascending=False).drop(["Community", "Shared genes", "Disease genes", "Community size"], axis=1)

In [48]:
disease_rank.head()

,Disease,Ratio disease,Ratio community,Relevance
1619,Intellectual Disability,0.278761,0.464945,0.129608
1614,Mental and motor retardation,0.361596,0.267528,0.096737
1617,Global developmental delay,0.345433,0.272140,0.094006
1616,Cognitive delay,0.364116,0.254613,0.092709
1622,Mental deficiency,0.336774,0.240775,0.081087


In [25]:
disease_rank.to_csv("datasets/community_gene_metrics.csv")

Compute the distances from a communitiy to the other ones.

In [26]:
def communities_distance(communities_rank: pd.DataFrame,
                         first_index: int,
                         second_index: int,
                         metric: str = "Relevance") -> float:
    # Retrieve all the diseases linked to the communities
    tmp_first = communities_rank[communities_rank["Community"] == first_index][["Disease", metric]]
    tmp_second = communities_rank[communities_rank["Community"] == second_index][["Disease", metric]]

    # Shared diseases between the two communities
    shared_diseases = tmp_first.merge(tmp_second, left_on="Disease", right_on="Disease")

    # Diseases that are not shared between the two communities
    tmp_first = tmp_first[~tmp_first['Disease'].isin(shared_diseases["Disease"])]
    tmp_second = tmp_second[~tmp_second['Disease'].isin(shared_diseases["Disease"])]

    distance = 0
    for _, disease in shared_diseases.iterrows():
        distance += np.power(disease[metric + "_x"] - disease[metric + "_y"], 2)

    for _, first_disease in tmp_first.iterrows():
        distance += np.power(first_disease[metric], 2)

    for _, second_disease in tmp_second.iterrows():
        distance += np.power(second_disease[metric], 2)

    return distance

In [27]:
son_comm = look_for_gene_community("SON", communities)
for i in range(len(communities)):
    print("Distance to community {0}: {1}".format(i, communities_distance(communities_rank, son_comm, i)))

Distance to community 0: 0.15309881692535318
Distance to community 1: 0.10621400794995371
Distance to community 2: 0.1509892632652227
Distance to community 3: 0.0
Distance to community 4: 0.14419228602325065
Distance to community 5: 0.14792731821379565
Distance to community 6: 0.13737694668498182
Distance to community 7: 0.08251813061038002
Distance to community 8: 0.11564524055172641


Save the communities into a csv file.

In [28]:
df_communities = pd.DataFrame(communities)
df_communities.to_csv("datasets/communities.csv")

In [29]:
son_index_diseases = protein_graph.nodes()['SON']['diseases']
son_diseases = df_diseases.loc[son_index_diseases, 'Term']

top_disc_dis = disease_rank[~disease_rank['Disease'].isin(son_diseases)].sort_values(by='Relevance', ascending=False)

In [30]:
top_disc_dis

,Disease,Ratio disease,Ratio community,Relevance
1615,Small head,0.412766,0.178967,0.073871
1965,Seizures,0.259804,0.195572,0.050810
1730,Malignant neoplasm of breast,0.125292,0.395756,0.049585
1842,Epilepsy,0.245763,0.187269,0.046024
1867,Breast Carcinoma,0.119269,0.367159,0.043791
...,...,...,...,...
2102,Renal fibrosis,0.080000,0.011070,0.000886
1790,Disseminated Malignant Neoplasm,0.077922,0.011070,0.000863
1912,Prostatic Intraepithelial Neoplasias,0.081481,0.010148,0.000827
1973,oligodendroglioma,0.078571,0.010148,0.000797


In [31]:
disease_rank[disease_rank['Disease'].isin(son_diseases)]

,Disease,Ratio disease,Ratio community,Relevance
1619,Intellectual Disability,0.278761,0.464945,0.129608
1614,Mental and motor retardation,0.361596,0.267528,0.096737
1617,Global developmental delay,0.345433,0.272140,0.094006
1616,Cognitive delay,0.364116,0.254613,0.092709
1622,Mental deficiency,0.336774,0.240775,0.081087
1624,Poor school performance,0.344130,0.235240,0.080953
1647,Mental Retardation,0.321513,0.250923,0.080675
1626,Dull intelligence,0.346910,0.227860,0.079047
1627,Low intelligence,0.346910,0.227860,0.079047
1620,Short stature,0.331034,0.177122,0.058633


In [34]:
df_diseases[df_diseases['Term']=='Seizures']

,Term,Overlap,P-value,Adjusted P-value,Genes
351,Seizures,816/1174,0.000486,0.013543,ATF2;JRK;NUP107;TSEN54;NSDHL;SYNGAP1;AKT2;AKT3...


In [33]:
look_for_gene_community("SON", communities)

3